In [16]:
%reload_ext autoreload
%autoreload 2
%matplotlib inline

import pandas as pd
import numpy as np
import lxml.etree as ET
from bs4 import BeautifulSoup
import os
import spacy
import string
import re
from tqdm.notebook import tqdm
import math
import numpy as np
from spacy.symbols import ORTH
from pathlib import Path
from collections import Counter
from collections import defaultdict
from sklearn.model_selection import train_test_split

import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
from torch.utils.data import Dataset, DataLoader
from torchvision import models
from torch.nn.utils.rnn import pack_padded_sequence, pad_packed_sequence

import cv2
import matplotlib.pyplot as plt

In [17]:
from google.colab import drive
drive.mount('/content/gdrive')

Drive already mounted at /content/gdrive; to attempt to forcibly remount, call drive.mount("/content/gdrive", force_remount=True).


## Test Images

In [ ]:
! pip install gdown

In [1]:
! gdown https://drive.google.com/uc?id=1E-B-9fBwmACjWYkFWpmgb971WSqWo7hd

Downloading...
From: https://drive.google.com/uc?id=1E-B-9fBwmACjWYkFWpmgb971WSqWo7hd
To: /content/GanTest_Images.zip
100% 268k/268k [00:00<00:00, 97.2MB/s]


In [2]:
! unzip GanTest_Images.zip > /dev/null

In [18]:
grootpath = Path('/content/gdrive/My Drive/Colab Notebooks/deep_learning/DoggyGANs')

In [19]:
train_df = pd.read_csv(grootpath/'generative-dog-images/files.csv')
train_df.head()

,f_name,f_subname,label
0,generative-dog-images/resized/n02109961_16718.jpg,n02109961_16718,Eskimo_dog
1,generative-dog-images/resized/n02113799_5009.jpg,n02113799_5009,standard_poodle
2,generative-dog-images/resized/n02091032_6587.jpg,n02091032_6587,Italian_greyhound
3,generative-dog-images/resized/n02106166_1429.jpg,n02106166_1429,Border_collie
4,generative-dog-images/resized/n02093754_2276.jpg,n02093754_2276,Border_terrier


In [20]:
gimage_path = Path(grootpath/'generative-dog-images/resized/')
train_df['g_name'] = train_df['f_name'].apply(lambda x: str(gimage_path) + '/' + x.split('/')[-1])
train_df.head()

,f_name,f_subname,label,g_name
0,generative-dog-images/resized/n02109961_16718.jpg,n02109961_16718,Eskimo_dog,/content/gdrive/My Drive/Colab Notebooks/deep_...
1,generative-dog-images/resized/n02113799_5009.jpg,n02113799_5009,standard_poodle,/content/gdrive/My Drive/Colab Notebooks/deep_...
2,generative-dog-images/resized/n02091032_6587.jpg,n02091032_6587,Italian_greyhound,/content/gdrive/My Drive/Colab Notebooks/deep_...
3,generative-dog-images/resized/n02106166_1429.jpg,n02106166_1429,Border_collie,/content/gdrive/My Drive/Colab Notebooks/deep_...
4,generative-dog-images/resized/n02093754_2276.jpg,n02093754_2276,Border_terrier,/content/gdrive/My Drive/Colab Notebooks/deep_...


In [21]:
train_df['colab_path'] = train_df['f_subname'].apply(lambda x: f"resized/{x}.jpg")
train_df.head()

,f_name,f_subname,label,g_name,colab_path
0,generative-dog-images/resized/n02109961_16718.jpg,n02109961_16718,Eskimo_dog,/content/gdrive/My Drive/Colab Notebooks/deep_...,resized/n02109961_16718.jpg
1,generative-dog-images/resized/n02113799_5009.jpg,n02113799_5009,standard_poodle,/content/gdrive/My Drive/Colab Notebooks/deep_...,resized/n02113799_5009.jpg
2,generative-dog-images/resized/n02091032_6587.jpg,n02091032_6587,Italian_greyhound,/content/gdrive/My Drive/Colab Notebooks/deep_...,resized/n02091032_6587.jpg
3,generative-dog-images/resized/n02106166_1429.jpg,n02106166_1429,Border_collie,/content/gdrive/My Drive/Colab Notebooks/deep_...,resized/n02106166_1429.jpg
4,generative-dog-images/resized/n02093754_2276.jpg,n02093754_2276,Border_terrier,/content/gdrive/My Drive/Colab Notebooks/deep_...,resized/n02093754_2276.jpg


In [22]:
label2index = {val:i for i, val in enumerate(sorted(set(train_df['label'])))}

## Making Predictions with Resnet Model

In [23]:
def load_model(m, p): 
  m.load_state_dict(torch.load(p))

In [24]:
class DogClassifier(nn.Module):
    def __init__(self, num_classes):
        super(DogClassifier, self).__init__()
        resnet = models.resnet34(pretrained=True)
        layers = list(resnet.children())[:8]
        self.features1 = nn.Sequential(*layers[:6])
        self.features2 = nn.Sequential(*layers[6:])
        self.classifier = nn.Sequential(nn.BatchNorm1d(512), nn.Linear(512, num_classes))
    
    def forward(self, x):
        x = self.features1(x)
        x = self.features2(x)
        x = F.relu(x)
        x = nn.AdaptiveAvgPool2d((1,1))(x)
        x = x.view(x.shape[0], -1)
        x = self.classifier(x)
        return x

In [25]:
num_classes = len(label2index)
model = DogClassifier(num_classes)

In [30]:
# load_model(model, str(grootpath/"ResnetClassifier"))
model = torch.load(grootpath/"ResnetClassifier.pth").cuda()
model.eval()

DogClassifier(
  (features1): Sequential(
    (0): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
    (1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (2): ReLU(inplace=True)
    (3): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
    (4): Sequential(
      (0): BasicBlock(
        (conv1): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
        (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        (relu): ReLU(inplace=True)
        (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
        (bn2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      )
      (1): BasicBlock(
        (conv1): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
        (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_sta

## Testing Images

In [31]:
class DogsDataset(Dataset):
    def __init__(self, df):
        self.df = df
        self.files = np.array(df['img_path'])

    def __len__(self):
        return self.df.shape[0]
    
    def __getitem__(self, idx):
        path = str(self.files[idx])
        x = cv2.imread(path).astype(np.float32)
        x = cv2.cvtColor(x, cv2.COLOR_BGR2RGB)/255
        return np.rollaxis(x, 2)

In [32]:
testpath = Path("GanTest_Images")
test_df = pd.DataFrame({'img_path':[file for file in list(testpath.iterdir()) if str(file).endswith(".jpg")]})

test_ds = DogsDataset(test_df)
batch_size = test_df.shape[0]
test_dl = DataLoader(test_ds, batch_size=batch_size)

In [35]:
def make_preds(model, test_dl):
    for x in test_dl:
      x = x.float().cuda()
      out = model(x)
      pred = torch.max(out, 1)[1]
    return pred

In [53]:
preds = make_preds(model, test_dl)
test_df['pred'] = preds.cpu().numpy()
index2label = {val:key for key, val in label2index.items()}
test_df['breed_pred'] = test_df['pred'].map(index2label)
test_df

,img_path,pred,breed_pred
0,GanTest_Images/GAN1.jpg,53,Pomeranian
1,GanTest_Images/GAN4.jpg,44,Maltese_dog
2,GanTest_Images/GAN9.jpg,44,Maltese_dog
3,GanTest_Images/GAN3.jpg,30,Great_Pyrenees
4,GanTest_Images/GAN2.jpg,44,Maltese_dog
5,GanTest_Images/GAN7.jpg,30,Great_Pyrenees
6,GanTest_Images/GAN10.jpg,44,Maltese_dog
7,GanTest_Images/GAN6.jpg,30,Great_Pyrenees
8,GanTest_Images/GAN5.jpg,58,Samoyed
9,GanTest_Images/GAN8.jpg,30,Great_Pyrenees


In [56]:
# test_df.to_csv(grootpath/'GAN_preds.csv')